# KPop song clustering

### Objective: Used unsupervised learning to cluster korean pop songs that I like (in playlist). Compare with Anime playlist. Compare with "iconic" KPop playlist

### Methodology: 1) Create dataset with track name, artist, audio features, playlist name, like y/n

In [1]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import time
import pandas as pd
import numpy as np

In [2]:
#Authorize credentials
#Note that with the free version, API calls are limited
cid = "c433ddf327d64bbcbae13b46dc5a56c9" 
secret = "ed9ffb3511ff4bb7910d520564ddbd79" 
user_id = '22pjtm23jl5h4qq4srfwxwxqa' #my playlists
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def getplayListDict(user_id):
    playlists = sp.user_playlists(user_id)
    names = [x['name'] for x in playlists['items']]
    ids = [x['id'] for x in playlists['items']]
    return dict(zip(names, ids))

In [4]:
def getTrackInfoFromPlaylist(user_id, playlist_id):
    offset = 0
    songs = []
    while True:
        content = sp.user_playlist_tracks(user_id, playlist_id, limit = 50, offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset+=50
        else:
            break
    names = [x['track']['name'] for x in songs]
    ids = [x['track']['id'] for x in songs]
    artists = [x['track']['artists'][0]['name'] for x in songs]
    df = pd.DataFrame({'Track Name': names, 'Track ID': ids, 'Artist': artists})
    return df

In [5]:
offset = 0
songs = []
items = []
ids = []
while True:
    content = sp.user_playlist_tracks(user_id, '3UeJumqoZFSXqRQ9b3F6SB', limit = 50, offset=offset)
    songs += content['items']
    if content['next'] is not None:
        offset+=50
    else:
        break

In [6]:
my_playlist = getplayListDict(user_id)

In [7]:
my_playlist

{'Kpop girl group songs that were a cultural reset': '3UeJumqoZFSXqRQ9b3F6SB',
 'KPOP': '75rPT4TszZleyYpYqzAZgf',
 'anime covers': '3duEwjd8M5Hugz8UkAFQdC',
 'lit anime music': '3LvTqHT3lOBomXlfqmUm8M',
 'Shounen Anime Vibes (Anime Opening & Ending)': '5MjsmYfqrFrg0AMGKhg1JI'}

In [8]:
pl_list = []
for playlist in my_playlist.keys():
    playlist_df = getTrackInfoFromPlaylist(user_id, my_playlist[playlist])
    playlist_df['Playlist'] = playlist
    pl_list.append(playlist_df)
tracks_df = pd.concat(pl_list)

In [9]:
#remove none
tracks_df.dropna(inplace=True)
tracks_df

,Track Name,Track ID,Artist,Playlist
0,Into the New World,24Gy6lVtCd8nOL2vzZiFyV,Girls' Generation,Kpop girl group songs that were a cultural reset
1,Tell Me,62x5q8OrmPpmlA8PpjrQt3,Wonder Girls,Kpop girl group songs that were a cultural reset
2,One More Time,1oD1B653Nje6Z0zoijB56I,Jewelry,Kpop girl group songs that were a cultural reset
3,So Hot,6ZizEG2wtzeHrgKEv1eBmD,Wonder Girls,Kpop girl group songs that were a cultural reset
4,Nobody,7C3JLAmQzy9SnTZjKcdzCA,Wonder Girls,Kpop girl group songs that were a cultural reset
...,...,...,...,...
97,自由の翼,0l6uRRIDPNw3OsmqCYkmvi,Linked Horizon,Shounen Anime Vibes (Anime Opening & Ending)
98,H.T,27MWlouE3EXEJUZbiAS6tk,Tsuneo Imahori,Shounen Anime Vibes (Anime Opening & Ending)
99,瞬間センチメンタル,53PJqUlPuzywePHDS7dm11,SCANDAL,Shounen Anime Vibes (Anime Opening & Ending)
100,THE HERO !! 〜怒れる拳に火をつけろ〜,4YycgLIc3P6oQRY5NiOybl,JAM Project,Shounen Anime Vibes (Anime Opening & Ending)


In [10]:
def getAudioFeatures(track_id_list, limit=50, sleeptime=10):
    features = []
    n_calls = 0
    for track_id in track_id_list:
        feature = sp.audio_features([track_id])
        features.append(feature)
        n_calls+=1
        if n_calls>50:
            time.sleep(sleeptime)
            n_calls = 0
    return features

In [11]:
def getAudioFeatures(track_id_list, limit=50):
    features = []
    index = 0
    while index < len(track_id_list):
        features += sp.audio_features(track_id_list[index:index+limit])
        index += limit
    return features

In [12]:
audio_features = pd.DataFrame(getAudioFeatures(tracks_df['Track ID'].tolist()))

In [16]:
audio_feature_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

In [17]:
df_merged_features = tracks_df.join(audio_features, how='inner', lsuffix='Track ID', rsuffix = 'id')

In [18]:
df_merged_features.to_csv('kpop_data_science/playlist_features.csv', index=None)